In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings(action = 'ignore')

In [2]:
print(fm.findSystemFonts(fontpaths=None, fontext='ttf'))

['/System/Library/Fonts/Supplemental/Telugu Sangam MN.ttc', '/System/Library/Fonts/Supplemental/NotoSansSaurashtra-Regular.ttf', '/System/Library/Fonts/Supplemental/NotoSansKayahLi-Regular.ttf', '/System/Library/Fonts/Supplemental/NotoSansGlagolitic-Regular.ttf', '/System/Library/Fonts/Supplemental/NotoSansNKo-Regular.ttf', '/System/Library/Fonts/Supplemental/PTSerifCaption.ttc', '/System/Library/Fonts/Supplemental/Diwan Kufi.ttc', '/System/Library/Fonts/Supplemental/Iowan Old Style.ttc', '/System/Library/Fonts/Supplemental/Khmer Sangam MN.ttf', '/Users/dy/miniforge3/envs/jedi/fonts/Ubuntu-LI.ttf', '/System/Library/Fonts/Supplemental/STIXNonUni.otf', '/System/Library/Fonts/Supplemental/Luminari.ttf', '/System/Library/Fonts/Supplemental/Courier New Bold.ttf', '/System/Library/Fonts/Supplemental/NotoSansPahawhHmong-Regular.ttf', '/System/Library/Fonts/KohinoorTelugu.ttc', '/System/Library/Fonts/Supplemental/Farisi.ttf', '/Users/dy/miniforge3/envs/jedi/fonts/SourceCodePro-ExtraLight.ttf',

In [3]:
f = [f.name for f in fm.fontManager.ttflist if 'Nanum' in f.name]
f


['NanumBarunGothic']

In [4]:
# import matplotlib as mpl
# mpl.get_configdir()

In [5]:
# import matplotlib.font_manager as fm

# fm._rebuild()

In [6]:
# 한글폰트 설정(셀 실행후 런타임 다시시작을 하여 해당셀을 다시한번 실행해주어야 제대로 한글이 그래프에 나옴) <-- jupyter note 재부팅 필수!!

# !apt -qq -y install fonts-nanum > /dev/null
# fontpath = '/Users/dy/Library/Fonts/NanumBarunGothic.ttf'
# font = fm.FontProperties(fname=fontpath, size=10)
# fm._rebuild()
plt.rc('font', family='NanumBarunGothic') 

# 그래프에 retina display 적용
%config InlineBackend.figure_format = 'retina'

# 버전확인
print('Pandas : %s'%(pd.__version__))
print('Numpy : %s'%(np.__version__))
print('sklearn : %s'%(sk.__version__))
print('matplotlib : %s'%(matplotlib.__version__))
print('seaborn : %s'%(sns.__version__))
!python --version

Pandas : 1.4.3
Numpy : 1.22.4
sklearn : 1.1.1
matplotlib : 3.5.2
seaborn : 0.11.2
Python 3.8.13


In [7]:
# 강의실 데이터 파일 불러오기
row_data = pd.read_csv('./22classData.csv')
print(row_data.shape)
row_data.tail()


(1071, 5)


,전공/학과,교과목명,담당교수,강의시간,수강인원
1066,실내디자인전공,인테리어디자인기초,한유진,화 E~F / N204,21
1067,시각디자인전공,시각디자인의기초,이승진,"목 E~F / , 목 E~F / N309",25
1068,시각디자인전공,시각디자인의기초,김승민,"목 E~F / , 목 E~F / N304",25
1069,텍스타일디자인전공,텍스타일디자인의기초,이재범,"화 C~D / , 화 C~D / N103",33
1070,텍스타일디자인전공,텍스타일디자인의기초,서윤정,화 C~D / 차324,32


In [8]:
row_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071 entries, 0 to 1070
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   전공/학과   1071 non-null   object
 1   교과목명    1071 non-null   object
 2   담당교수    1071 non-null   object
 3   강의시간    1030 non-null   object
 4   수강인원    1071 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 42.0+ KB


In [9]:
row_data.isnull().sum()

전공/학과     0
교과목명      0
담당교수      0
강의시간     41
수강인원      0
dtype: int64

In [10]:
row_data = row_data.replace(0, np.NaN)

In [11]:
row_data=row_data.dropna()
row_data

,전공/학과,교과목명,담당교수,강의시간,수강인원
0,글로벌커뮤니케이션센터(학사),Advertising Communication principles in Digita...,앤써니캐셔(Anthony Kaschor),"화 C / 차121, 목 E / 차121",37.0
1,글로벌커뮤니케이션센터(학사),English for Career Development,마크샘플(Mark Sample),"화 A / 차123, 목 B / 차123",33.0
2,글로벌커뮤니케이션센터(학사),English for Global Tourism,허켈리(Kelly Un Kyong Hur),"화 C / 차122, 목 E / 차122",27.0
3,글로벌커뮤니케이션센터(학사),Introduction to Women’s Sexuality,쉐리슬릭(Sheri Slick),"화 A / , 목 B /",50.0
4,글로벌커뮤니케이션센터(학사),Korean Economy and South East Asian Markets,Haggai Kennedy Ochieng,"월 F / 차227, 수 D / 차227",6.0
...,...,...,...,...,...
1066,실내디자인전공,인테리어디자인기초,한유진,화 E~F / N204,21.0
1067,시각디자인전공,시각디자인의기초,이승진,"목 E~F / , 목 E~F / N309",25.0
1068,시각디자인전공,시각디자인의기초,김승민,"목 E~F / , 목 E~F / N304",25.0
1069,텍스타일디자인전공,텍스타일디자인의기초,이재범,"화 C~D / , 화 C~D / N103",33.0


In [12]:
row_data.isnull().sum()

전공/학과    0
교과목명     0
담당교수     0
강의시간     0
수강인원     0
dtype: int64

In [24]:
import re
from collections import defaultdict

row_data['강의시간']=row_data['강의시간'].str.replace(" ","")

lec_list=[]

classroom=defaultdict(set)

# def define_lec_time():
for str in (row_data['강의시간']):
#     print(str)
    str.split(',')
    # 강의시간의 요소가 2개일 경우, 쪼갠다.
    if len(str.split(',')) == 2:
#       str = re.split('[,|/]', str)  # , or / 분리
        str = re.split(',', str) # 콤마 기준으로 나뉨
        #print('str is...', str)
        test = []
        for i in str:
            #print('i is...', i)
            i=re.split('/', i) # / 분리
            for k in i:
                test.append(k)
        # print('test is...', test) 
        if len(test)>3:
            if test[1]==test[3]:
                classroom[test[1]].add(test[0]) # .add()로 더할것
                classroom[test[1]].add(test[2])
            elif test[1]!=test[3]:
                classroom[test[1]].add(test[0])
                classroom[test[3]].add(test[1])
        elif len(test)==3: 
            print(test)
            classroom[test[len(test)-1]].add(test[0])
            classroom[test[len(test)-1]].add(test[0][0]+test[1]) # 앞에 요일 추가함
        elif len(test)==2:
            if test[0] != test[1]: # 금 E, F / 인326
                str = re.split('/', test[1])
                yeongang=[]
                for i in str:
                    yeongang.append(i)
                classroom[yeongang[1]].add(test[0])
                classroom[yeongang[1]].add(yeongang[0])
            
    elif len(str.split(','))==1:
        str = re.split('/', str)
        test2=[]
        for i in str:
            test2.append(i)
        classroom[test2[1]].add(test2[0])
        
df = pd.DataFrame(classroom.items(), columns=['room', 'time'])

df.to_csv('./preprocessing.csv', encoding='utf-8-sig')
print('preprocessing data done!@!!')

['금E', 'F', '인326']
['화A', 'B', '차124']
['금A', 'B', '인328']
['금C', 'D', '인328']
preprocessing data done!@!!


In [15]:

# return lec_list
        
# '''
#     ①월A/수B   ②월B/수A   ③월C/수E   ④월D/수F   ⑤월E/수C

# ⑥월F/수D   ⑦화A/목B   ⑧화B/목A   ⑨화C/목E   ⑩화D/목F

# ⑪화E/목C    ⑫화F/목D    ⑬금A~B   ⑭금C~D    ⑮금E~F
# '''    
# cha_class=[]
# in_class=[]
# big_class=[]
# ja_class=[]
# yu_class=[]
# yak_class=[]
# f_class=[]
# l_class=[]
# n_class=[]
# k_class=[]
# la_class=[]
# nu_class=[]
# etc_class=[]

# cha_room = defaultdict(lambda: '0')

# for j in define_lec_time():
#     # print(j) # ['화C', '차121', '목E', '차121']
#     for i in j:
#         if '차' in i:
#             cha_room[i]=j
#     for k in j:
#         if '차' in k:
#             cha_class.append(k)
#         elif '인' in k:
#             in_class.append(k)
#         elif '대' in k:
#             big_class.append(k)
#         elif '자' in k:
#             ja_class.append(k)
#         elif '유' in k:
#             yu_class.append(k)
#         elif '약' in k:
#             yak_class.append(k)
#         elif 'F' in k:
#             f_class.append(k)
#         elif 'L' in k:
#             l_class.append(k)
#         elif 'N' in k:
#             n_class.append(k)
#         elif 'K' in k:
#             k_class.append(k)
#         elif '라' in k:
#             la_class.append(k)
#         elif '누' in k:
#             nu_class.append(k)
#         else:
#             etc_class.append(k)
        
#     if '월A' in j and '수B' in j:
#         print(j)

In [ ]:
# time의 강의실은 빼고 싶다
time_list=[]
for i in range(len(df)):
    for j in (df['time'][i]):
        print(j)
        if '차' in j:
            pass
        else:
            time_list.append(j)
print(time_list)
        


In [ ]:
for j in define_lec_time():
#     print(j)
    for k in j:
        print('-'*20)
        for c in cha_class:
#             print(c)
            if c ==k:
                print(c)
        

In [ ]:
for c in cha_class:
    print(c)

In [ ]:
cha_class

In [ ]:
in_class

In [ ]:
etc_class